&emsp; 3 &ensp; CoLLiE 的并行策略

&emsp; &emsp; 3.1 &ensp; 数据并行（Data Parallel）

&emsp; &emsp; 3.2 &ensp; 流水线并行（Pipeline Parallel）

&emsp; &emsp; 3.3 &ensp; 张量并行（Tensor Parallel）

&emsp; &emsp; 3.4 &ensp; 整合：3D并行（3D Parallel）

&emsp; &emsp; 3.5 &ensp; 零冗余优化器（ZeRO）

## 3. &ensp; CoLLiE 的并行策略

&ensp; &ensp; 本文将详细介绍CoLLiE使用的并行策略，并行策略是让模型训练突破单张显卡计算和存储上限的不同方法，CoLLiE支持的并行策略包括：数据并行 DP、流水线并行 PP、张量并行 TP、3D并行（对上述三者的整合）、零冗余优化器 ZeRO，其简要特征如下表所示。

| 名称 | 简称 | 切分数据 | 切分模型 | 切分梯度 | 切分优化器状态 | 补充说明 |
|:----|:----|:----:|:----:|:----:|:----:|:----|
| 数据并行 | DP | ✔ | ✘ | ✘ | ✘ | 每个卡保留全部的模型参数、回传梯度、优化器状态 |
| 流水线并行 | PP | ✘ | 横向切分 | — | — | 存在 bubble time 的问题，梯度和优化器状态随模型参数切分 | 
| 张量并行 | TP | ✘ | 纵向切分 | — | — | 注意 先列切分 再行切分，梯度和优化器状态随模型参数切分 |
| （整合）| 3D | ✔ | 切分两次 | — | — | 初始先PP 再TP，通过在dp_size张卡保留相同模型切分，处理不同数据 |
| 零冗余优化器 | ZeRO1 | ✔ | ✘ | ✘ | ✔ | 相比DP不额外增加通信，对于Adam等优化算法能显著降低优化器状态存储 |
|  | ZeRO2 | ✔ | ✘ | ✔ | ✔ | 相比DP不额外增加通信，在ZeRO1基础上切分梯度 | 
|  | ZeRO3 | ✔ | ✔ | ✔ | ✔ | 在ZeRO2基础上切分模型，每张卡都保留不同模型切分，处理不同数据 |


&ensp; &ensp; **数据并行 DP**，将不同batch的数据分配给不同的卡，而让每个卡保留全部的完整模型，实现了分布式训练；虽然提升了模型训练效率，但是在存储上存在巨大的冗余，无法解决单张卡装不下整个大模型的问题。

&ensp; &ensp; **流水线并行 PP**，利用当前大模型由若干相同的Transformer层组成的特点，先将模型横向切分，再将参数分摊到不同的卡上；虽然缓解了DP存储冗余的问题，但在计算时存在 bubble time 问题（后文有详细分析和图示）。

&ensp; &ensp; **张量并行 TP**（有些文献中也直接将之称为模型并行）则另辟蹊径，选择将模型横向切分，即让每张卡都具有模型每一层的每一切片，对于Transformer模型的每一部分，TP都给予了特定的切分方法，整体上大致遵循“先列切分，再行切分”的策略。

&ensp; &ensp; DP、PP、TP 彼此之间是可以相互兼容促进的，三者的整合被称作**3D并行**。

&ensp; &ensp; 

### 3.1 &ensp; 数据并行（Data Parallel）

&ensp; &ensp; Distributed Data Parallel（DDP），

<br>
<!-- <table style="background:none">
  <tr>
    <th><img src="images/data-parallel-forward.png" width="100%" height="100%" align="center" /></th>
    <th><img src="images/data-parallel-backward.png" width="100%" height="100%" align="center" /></th>
  </tr>
</table> -->
<div>
  <div style="width:50%;float:left;"><img src="images/data-parallel-forward.png" width="86%" height="86%" align="center" /></div>
  <div style="width:50%;float:left;"><img src="images/data-parallel-backward.png" width="86%" height="86%" align="center" /></div>
</div>

### 3.2 &ensp; 流水线并行（Pipeline Parallel）

&ensp; &ensp; 横向切分，将不同的层放到不同的设备上；设备之间存在数据依赖，依次执行，存在 bubble；设备间通信较小

<br>
<div>
  <div style="width:50%;float:left;"><img src="images/pipeline-parallel.png" width="60%" height="60%" align="center" /></div>
  <div style="width:50%;float:left;"></div>
</div>

### 3.3 &ensp; 张量并行（Tensor Parallel）

&ensp; &ensp; 在batch_size较小时相较于流水线并行优势更加明显

纵向切分，mlp和selfattn的切法不同；mlp部分，gelu前的要split column-wise，gelu后的要split row-wise，可以想想象为multi-way mlp，避免切分后中间同步相加；selfattn部分，QKV要split column-wise，本质上就是multi-head，O要split row-wise，同样避免中间同步相加；其他部分：input_embed要split column-wise，切分出多个片段，output_embed要split row-wise，将多个片段聚合，但为了降低开销，只聚合loss。

activation checkpointing：简单地说，不要存储transformer中每层的激活值，每若干层存一份；回传过程中，如果该层没有存储，就从最近的层前向传输过来

相比于PP，需要server之间通信，TP尽量做到server内部通信；流水线并行PP和张量并行TP相互兼容，同时也兼容数据并行DP，合在一起就是3D并行 3D Parallel（TP本身根据切分维度也有2d-tp等，默认1d-tp）

<br>
<div>
  <div style="width:60%;float:left;"><img src="images/tensor-parallel.png" width="86%" height="86%" align="center" /></div>
  <div style="width:40%;float:left;"></div>
</div>

### 3.4 &ensp; 整合：3D并行（3D Parallel）

### 3.5 &ensp; 零冗余优化器（ZeRO）  


ZeRO包括两个部分：ZeRO-DP，ZeRO-powered data parallel，ZeRO-R，负责优化冗余状态存储；其中，ZeRO-DP包括三个阶段：ZeRO1，切分优化器状态 state partitioning，ZeRO2，增加切分梯度 add gradient partitioning（以上相比数据并行不额外增加通信），ZeRO3，增加模型切分 add parameter partitioning

ZeRO3是数据并行和模型并行的结合（ZeRO不涉及具体的模型并行方式），每个卡负责不同的数据，并且负责不同的模型；数据前向计算到哪一层，就把那一层参数传过来，然后释放掉（如果那一层自己有就不用传输，不用释放）；损失反向回传到哪一层，就把那一层梯度传过来，释放前再返回那一层所在的卡将梯度聚合，并且根据那张卡上有的优化器状态，更新参数

论及冗余状态存储（residual state memory，与model state memory相对，即参数 梯度 优化器的存储），ZeRO-R涉及三个方面：激活值 activation，activation checkpointing & partitioning；缓存区 temporary buffers，定长缓存区；碎片管理 memory fragmentation，反向传播时，激活值的checkpoint和参数的梯度存储较长，重计算的激活值和激活值的梯度存储较短，前者放在连续缓存区，避免和后者交叉使用产生碎片

<br>
<!-- <table style="background:none">
  <tr>
    <th><img src="images/zero-forward.png" width="100%" height="100%" align="center" /></th>
    <th><img src="images/zero-backward.png" width="100%" height="100%" align="center" /></th>
  </tr>
</table> -->
<div>
  <div style="width:50%;float:left;"><img src="images/zero-forward.png" width="86%" height="86%" align="center" /></div>
  <div style="width:50%;float:left;"><img src="images/zero-backward.png" width="86%" height="86%" align="center" /></div>
</div>